In [ ]:
!pip install requests --quiet
!pip install beautifulsoup4 --quiet
!pip install pandas --quiet
!pip install datetime --quiet

**Imports**

In [1]:
import pandas as pd
import numpy as np
import requests
import datetime
from bs4 import BeautifulSoup

**Functions**

In [96]:
def read_metadata(filename):
    '''
    reads in metadata from the api.py file and adds an empty column where the content of the pages will be in the end
    '''
    data = pd.read_csv(filename)
    data['Content'] = None
    return data

def get_url(cellar_ref, doctype="03"):
    '''
    creates a url based on the cellar reference in the metadata, which will be used to scrape the content
    '''
    psid = cellar_ref
    psname = "cellar" # other options: cellar, celex, oj...
    lancode = "0006" # language code
    doctype = doctype # default: 03
    docnum = "DOC_1"
    # for further information, see Documentation Page 37: https://op.europa.eu/en/publication-detail/-/publication/50ecce27-857e-11e8-ac6a-01aa75ed71a1/language-en/format-PDF/source-73059305
    return f"http://publications.europa.eu/resource/{psname}/{psid}.{lancode}.{doctype}/{docnum}"

def get_content(URL):
    '''
    main function, scrapes content. added some code to catch errors.
    '''
    response = requests.get(URL, headers={"Accept-Language":"en-US"})
    # one minor bug still in there: some requests (for example number 58 in 20220601_larger_data_b) are a valid request but have to download many mb first. the solution would be to stop the request.get if it runs longer than x seconds
    try:
        soup = BeautifulSoup(response.content, "html.parser")
        if str(soup)[1:4] == "PDF":
            '''
            in some (few) cases, the doctype is not 03 but 02. change it for these cases
            '''
            print("pdf detected, but fixed")
            doctype = '02'
            URL = URL[:-8] + doctype + URL[-6:]
            response = requests.get(URL, headers={"Accept-Language":"en-US"})
            soup = BeautifulSoup(response.content, "html.parser")
        else:
            print("no problem here")
            doctype = '03'
    except:
        '''
        in case there is an error
        '''
        print("yes problem here")
        URL = URL[:-8] + '02' + URL[-6:]
        response = requests.get(URL, headers={"Accept-Language":"en-US"})
        soup = BeautifulSoup(response.content, "html.parser")
        
    if soup.find("p", class_="oj-normal") == None:
        content = ' '.join([item.text for item in soup.find_all("p", class_="normal")])
    else:
        content = ' '.join([item.text for item in soup.find_all("p", class_="oj-normal")])
    return content

def clean_data(data):
    '''
    takes scraped data and removes rows which contain, no information, information in non-english and the head of all the valid content
    '''
    data = data[data['Content'] != ""]
    data = data[data['Content'].str[0:3] == 'THE'] #remove content in other languages
    data = data[data['Content'].str.contains('Whereas: ')] # contains the split word
    data.loc[:, 'Content'] = data['Content'].apply(lambda x: x.split('Whereas: ', 1)[1]) # split off header
    data = data[data['Content'].str[0:3] == "(1)"] #gotta make sure it's standardized!
    return data.reset_index().drop(columns = "index")

def get_all_content(data):
    '''
    loops over the functions to get all content
    '''
    cellar_references = data['cellar']    
    for index, ref in enumerate(cellar_references):
        data.loc[index, 'Content'] = get_content(get_url(ref))
        print(f'Row {index} with cellar-number {ref} done')
    return data

def get_all_content_with_splitting(data, batchsize=500, path ="../raw_data/", filename_without_csv="20220602"):
    '''
    loops over the functions to get all content. for more than 500 files the process will be split up
    '''
    if len(data) > batchsize:
        remaining = len(data) % batchsize
        iterations = int((len(data)-remaining) / batchsize)
        for batch in range(0,iterations):
            tmp = data.iloc[batch*batchsize:(batch+1)*batchsize]
            cellar_references = tmp['cellar'] 
            for index, ref in enumerate(cellar_references):
                tmp.loc[index, 'Content'] = get_content(get_url(ref))
                print(f'Row {index} from batch {batch} with cellar-number {ref} done')
                tmp.to_csv(f"{path}{filename_without_csv}_tmp_batch{batch}.csv")
        # add remaining rows
        #return "partitioned data can be found in '../raw_data/'
    else:
        cellar_references = data['cellar']    
        for index, ref in enumerate(cellar_references):
            data.loc[index, 'Content'] = get_content(get_url(ref))
            print(f'Row {index} with cellar-number {ref} done')
        return data

**Workflow**

In [ ]:
%%time
#retrieve metadata
path = "../raw_data/"
filename_without_csv = "20220602"
data = read_metadata(path + filename_without_csv + '.csv')

# subset metadata
data = data.iloc[8000:] 
# started at 12:00 at 8000

# get content
data_with_content = get_all_content_with_splitting(data, batchsize = 500)

# clean content
#data_with_content_clean = clean_data(data_with_content)

# export data to csv
#data_with_content_clean.to_csv(path + filename_without_csv + "_scraped_test.csv")

no problem here
Row 0 from batch 0 with cellar-number a9754cd1-e50a-4a3b-becb-704873aba01d done


/tmp/ipykernel_18117/1903783764.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp.loc[index, 'Content'] = get_content(get_url(ref))


no problem here
Row 1 from batch 0 with cellar-number 700cf946-b6ac-11ea-bb7a-01aa75ed71a1 done
no problem here
Row 2 from batch 0 with cellar-number 54b35a83-cd0a-11ec-a95f-01aa75ed71a1 done


/home/jakobg/.pyenv/versions/3.8.12/envs/BatchLegal/lib/python3.8/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


no problem here
Row 3 from batch 0 with cellar-number 0091c5ed-9d1a-11e2-ab01-01aa75ed71a1 done
no problem here
Row 4 from batch 0 with cellar-number 6fcc7323-9d00-11e7-b92d-01aa75ed71a1 done
no problem here
Row 5 from batch 0 with cellar-number 6914afe7-e9fc-11e2-99b0-01aa75ed71a1 done
no problem here
Row 6 from batch 0 with cellar-number bd856048-7547-11e4-b593-01aa75ed71a1 done
no problem here
Row 7 from batch 0 with cellar-number aae0d092-37f6-11e4-8c3c-01aa75ed71a1 done
no problem here
Row 8 from batch 0 with cellar-number 38e850de-d174-11e7-a5b9-01aa75ed71a1 done
no problem here
Row 9 from batch 0 with cellar-number 1ff55a20-14b4-11e5-8817-01aa75ed71a1 done
no problem here
Row 10 from batch 0 with cellar-number c43335ba-5e00-11e6-9b08-01aa75ed71a1 done
no problem here
Row 11 from batch 0 with cellar-number a8d3732d-a1f7-11e2-ab01-01aa75ed71a1 done


...

/home/jakobg/.pyenv/versions/3.8.12/envs/BatchLegal/lib/python3.8/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


no problem here
Row 147 from batch 0 with cellar-number b21bf6b3-c518-11e2-ab01-01aa75ed71a1 done


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


pdf detected, but fixed
Row 148 from batch 0 with cellar-number cf46df1b-e20d-11e7-9749-01aa75ed71a1 done
no problem here
Row 149 from batch 0 with cellar-number 70751579-5122-11e9-a8ed-01aa75ed71a1 done
no problem here
Row 150 from batch 0 with cellar-number 7705f6fa-423b-11e3-ae03-01aa75ed71a1 done
no problem here
Row 151 from batch 0 with cellar-number 263e49b3-876a-11e9-9f05-01aa75ed71a1 done
no problem here
Row 152 from batch 0 with cellar-number bd4772d3-23ce-11e6-86d0-01aa75ed71a1 done
no problem here
Row 153 from batch 0 with cellar-number 563bbbf5-47c6-11e5-9f5a-01aa75ed71a1 done
no problem here
Row 154 from batch 0 with cellar-number 18d1c66a-37d6-11e8-b5fe-01aa75ed71a1 done
no problem here
Row 155 from batch 0 with cellar-number bde0320b-55d1-11e4-a0cb-01aa75ed71a1 done
no problem here
Row 156 from batch 0 with cellar-number 9fd3fc1f-ad78-11e7-837e-01aa75ed71a1 done
no problem here
Row 157 from batch 0 with cellar-number 7f792e2c-56b7-11e5-afbf-01aa75ed71a1 done
no problem h

KeyboardInterrupt: 

**Read partitioned Data**

In [88]:
data_full = pd.read_csv("../raw_data/tmp/20220602_tmp_batch0.csv")

liste = list(np.arange(1,15)) # number+1 of tmp-dataframes, minus the first one

for i in liste:
    data_tmp = pd.read_csv(f"../raw_data/tmp/20220602_tmp_batch{i}.csv")
    data_tmp['Content'].iloc[0:499] = data_tmp['Content'].iloc[500:999].reset_index()['Content']
    data_tmp = data_tmp.drop(list(np.arange(500,1000)))
    data_full = data_full.append(data_tmp)
    
data_full = data_full.reset_index().drop(columns = ['Unnamed: 0.1', 'Unnamed: 0', 'index'])
data_full = clean_data(data_full)
#data_full.to_csv("../raw_data/20220602_part1_scraped.csv") # save to csv

# drop duplicates!

/tmp/ipykernel_18117/104240831.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_tmp['Content'].iloc[0:499] = data_tmp['Content'].iloc[500:999].reset_index()['Content']
/tmp/ipykernel_18117/104240831.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_full = data_full.append(data_tmp)
/tmp/ipykernel_18117/104240831.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_tmp['Content'].iloc[0:499] = data_tmp['Content'].iloc[500:999].reset_index()['Content']
/tmp/ipykernel_18117/104240831.py:7: FutureWarning: The frame.a

**Test Area**

In [ ]:
get_url(data['cellar'][100])